In [12]:
pip install mediapipe

  Using cached protobuf-4.25.6-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.6-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

def VirtualCoorMediaPipe(image):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    excluded_points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,31, 32]
    landmarks_list = []

    if image is None:
        print("Error: Image not found or cannot be loaded.")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        for idx, landmark in enumerate(result.pose_landmarks.landmark):
            x = int(landmark.x * image.shape[1])
            y = int(landmark.y * image.shape[0])
            if idx not in excluded_points:
                print(f"point {idx}: coordinates ({x:.1f}, {y:.1f})")
                landmarks_list.append([x, y, 1])  # Adding homogeneous coordinate

                cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(image, str(idx), (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        cv2_imshow(image)
    else:
        print("Landmarks pts NOT found.")

    pose.close()
    return np.array(landmarks_list)  # Return landmark points

# Load the image
image = cv2.imread('/content/T2PA1.jpg')
Q1 = VirtualCoorMediaPipe(image)

if Q1 is not None:
    # Transformation parameters for poster A
    R1 = np.array([[ 1.        ,  0.        ,  0.        ],
    [ 0.        , -0.07635921, -0.99708037],
    [ 0.        , -0.99708037,  0.07635921]])


    T1 = np.array([[  0.        ],
    [656.63923429],
    [297.14647033]])

    scale_factor_A = 0.186

    def calculate_physical_coordinates(R, T, Q, scale_factor):
      R_inv = np.linalg.inv(R)
      Q_scaled = Q * scale_factor
      Q_translated = Q_scaled.T - T
      P = (R_inv @ Q_translated).T
      return P

    P1 = calculate_physical_coordinates(R1, T1, Q1, scale_factor_A)
    print("Physical Coordinates:")
    print(P1)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

def VirtualCoorMediaPipe(image):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    excluded_points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32]
    landmarks_list = []

    if image is None:
        print("Error: Image not found or cannot be loaded.")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        for idx, landmark in enumerate(result.pose_landmarks.landmark):
            x = int(landmark.x * image.shape[1])
            y = int(landmark.y * image.shape[0])
            if idx not in excluded_points:
                print(f"point {idx}: coordinates ({x:.1f}, {y:.1f})")
                landmarks_list.append((x, y, 0))  # Convert to required format

                cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
                cv2.putText(image, str(idx), (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

        cv2_imshow(image)
    else:
        print("Landmarks pts NOT found.")

    pose.close()
    return np.array(landmarks_list)  # Return landmark points in required format

# Load the image
image = cv2.imread('/content/T2PB1.jpg')
Q2 = VirtualCoorMediaPipe(image)

if Q2 is not None:
    # Transformation parameters for poster B
    R2 = np.array([[ 0.99999162, -0.00409385,  0.        ],
    [-0.00409385, -0.99999162,  0.        ],
    [ 0.        ,  0.        ,  1.        ]])



    T2 = np.array([[101.70790574],
    [356.35999649],
    [  0.        ]])



    scale_factor_B = 0.183

    def calculate_physical_coordinates(R, T, Q, scale_factor):
      R_inv = np.linalg.inv(R)
      Q_scaled = Q * scale_factor
      Q_translated = Q_scaled.T - T
      P = (R_inv @ Q_translated).T
      return P

    P2 = calculate_physical_coordinates(R2, T2, Q2, scale_factor_B)
    print("Physical Coordinates:")
    print(P2)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# P1 = calculate_physical_coordinates(R1, T1, Q1, scale_factor_A)
# P2 = calculate_physical_coordinates(R2, T2, Q2, scale_factor_B)

# Fusion: Take X from poster B, Y and Z from poster A
fused_coordinates = np.column_stack((P2[:, 0], P1[:, 1], P1[:, 2]))


print("Fused 3D Coordinates:\n", fused_coordinates)

Fused 3D Coordinates:
 [[140.04030528 331.43457987 438.80028608]
 [129.52398679 330.39777451 425.26192873]
 [110.49139709 329.98589292 419.88367718]
 [143.7141595  325.49780397 361.27928095]
 [111.44949213 324.94389425 354.0464599 ]
 [123.29759421 318.12654394 265.02712386]
 [ 60.17585923 319.80247589 286.91104397]]


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

def VirtualCoorMediaPipe(image, order='A'):
    """
    Extracts pose landmarks from an image using MediaPipe and formats them.

    Parameters:
        image: Input image.
        order:
          'A' -> returns coordinates as (0, x, y)
          'B' -> returns coordinates as (x, y, 0)

    Returns:
        An (N, 3) NumPy array of landmarks in the specified format.
    """
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5,
                        min_tracking_confidence=0.5)

    # Exclude specific landmark indices (as provided earlier)
    excluded_points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                       13, 14, 17, 18, 19, 20, 21, 22, 23,
                       24, 25, 26, 27, 28, 31, 32]
    landmarks_list = []

    if image is None:
        print("Error: Image not found or cannot be loaded.")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        for idx, landmark in enumerate(result.pose_landmarks.landmark):
            # Compute pixel coordinates
            x = int(landmark.x * image.shape[1])
            y = int(landmark.y * image.shape[0])
            if idx not in excluded_points:
                if order == 'A':
                    # Format for poster A: (0, x, y)
                    landmarks_list.append((0, x, y))
                elif order == 'B':
                    # Format for poster B: (x, y, 0)
                    landmarks_list.append((x, y, 0))
                else:
                    landmarks_list.append((x, y, 0))  # default
    else:
        print("Landmarks not found.")

    pose.close()
    return np.array(landmarks_list)

# Load images (make sure the image paths are correct)
image_A = cv2.imread('/content/A_with_person.jpg')
image_B = cv2.imread('/content/B_with_person.jpg')

# Extract virtual coordinates in required formats
Q1 = VirtualCoorMediaPipe(image_A, order='A')  # For poster A: (0, x, y)
Q2 = VirtualCoorMediaPipe(image_B, order='B')  # For poster B: (x, y, 0)

if Q1 is not None and Q2 is not None:
    # Transformation parameters for poster A
    R1 = np.array([[1.0, 0.0, 0.0],
                   [0.0, 0.00966169, -0.99995332],
                   [0.0, -0.99995332, -0.00966169]])
    T1 = np.array([[0.0],
                   [309.86320746],
                   [270.60082593]])
    scale_factor_A = 0.083

    # Transformation parameters for poster B
    R2 = np.array([[0.99901468, 0.04438102, 0.0],
                   [0.04438102, -0.99901468, 0.0],
                   [0.0, 0.0, 1.0]])
    T2 = np.array([[44.62353032],
                   [312.68703378],
                   [0.0]])
    scale_factor_B = 0.0923

    def calculate_physical_coordinates(R, T, Q, scale_factor):
        """
        Converts virtual coordinates Q (shape: [N, 3]) into physical coordinates.

        The conversion applies scaling, translation, and rotation.
        """
        R_inv = np.linalg.inv(R)
        Q_scaled = Q * scale_factor  # Scale each coordinate
        Q_translated = Q_scaled.T - T  # Subtract translation vector (broadcasted)
        P = (R_inv @ Q_translated).T  # Apply rotation and transpose back
        return P

    # Calculate physical coordinates for each poster
    P1 = calculate_physical_coordinates(R1, T1, Q1, scale_factor_A)
    P2 = calculate_physical_coordinates(R2, T2, Q2, scale_factor_B)

    # Fusion: Take X from poster B, Y and Z from poster A
    fused_coordinates = np.column_stack((P2[:, 0], P1[:, 1], P1[:, 2]))

    print("Fused 3D Coordinates(normal):")
    print(fused_coordinates)

    print("===============")

    fused_coordinates2 = np.column_stack((P2[:, 0], (P1[:, 1] + P2[:, 1])/2 , P1[:, 2]))
    print("Fused 3D Coordinates(Mean):")
    print(fused_coordinates2)



Fused 3D Coordinates(normal):
[[ 54.11914577 234.97256144 132.1956104 ]
 [ 83.13558944 204.65292078 108.82758061]
 [ 28.79578181 203.5266526  130.8957291 ]
 [ 86.36135703 124.96821145 134.70189946]
 [ 28.73479437 133.34359769 152.71166054]]
Fused 3D Coordinates(Mean):
[[ 54.11914577 251.39542241 132.1956104 ]
 [ 83.13558944 218.49431092 108.82758061]
 [ 28.79578181 217.0475281  130.8957291 ]
 [ 86.36135703 139.22644093 134.70189946]
 [ 28.73479437 142.08791448 152.71166054]]


In [18]:
import mediapipe as mp
import cv2
import numpy as np

print("MediaPipe version:", mp.__version__)
print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)


MediaPipe version: 0.10.21
OpenCV version: 4.11.0
NumPy version: 1.26.4


In [1]:
pip install mediapipe==0.10.9

In [ ]:
#####Use this code to calculate final physical coordinates

In [2]:
import cv2
import mediapipe as mp
import numpy as np

class PoseFusion:
    def __init__(self, image_A, image_B):
        self.image_A = image_A
        self.image_B = image_B
        self.mp_pose = mp.solutions.pose
        self.excluded_points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                13, 14, 17, 18, 19, 20, 21, 22, 23,
                                24, 25, 26, 27, 28, 31, 32]

    def extract_landmarks(self, image, order='A'):
        """ Extracts and formats pose landmarks from an image using MediaPipe. """
        pose = self.mp_pose.Pose(min_detection_confidence=0.5,
                                 min_tracking_confidence=0.5,
                                 )

        if image is None:
            print("Error: Image not found or cannot be loaded.")
            return None

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)
        landmarks_list = []

        if result.pose_landmarks:
            for idx, landmark in enumerate(result.pose_landmarks.landmark):
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                if idx not in self.excluded_points:
                    if order == 'A':
                        landmarks_list.append((0, x, y))  # (0, x, y)
                    elif order == 'B':
                        landmarks_list.append((x, y, 0))  # (x, y, 0)
                    else:
                        landmarks_list.append((x, y, 0))  # Default
        else:
            print("Landmarks not found.")

        pose.close()
        return np.array(landmarks_list)

    def calculate_physical_coordinates(self, R, T, Q, scale_factor):
        """ Converts virtual coordinates to physical coordinates using scaling, rotation, and translation. """
        R_inv = np.linalg.inv(R)
        Q_scaled = Q * scale_factor
        Q_translated = Q_scaled.T - T  # Broadcast subtraction
        return (R_inv @ Q_translated).T

    def fuse_coordinates(self):
        """ Processes images, extracts landmarks, and fuses 3D coordinates. """
        Q1 = self.extract_landmarks(self.image_A, order='A')  # (0, x, y)
        Q2 = self.extract_landmarks(self.image_B, order='B')  # (x, y, 0)

        if Q1 is None or Q2 is None:
            return None

        # Transformation parameters for poster A
        R1 = np.array([[ 1.        ,  0.        ,  0.        ],
        [ 0.        , -0.07635921, -0.99708037],
        [ 0.        , -0.99708037,  0.07635921]])


        T1 = np.array([[  0.        ],
        [656.63923429],
        [297.14647033]])

        scale_factor_A = 0.186

        # Transformation parameters for poster B
        R2 = np.array([[ 0.99999162, -0.00409385,  0.        ],
        [-0.00409385, -0.99999162,  0.        ],
        [ 0.        ,  0.        ,  1.        ]])



        T2 = np.array([[101.70790574],
        [356.35999649],
        [  0.        ]])



        scale_factor_B = 0.183

        # Compute physical coordinates
        P1 = self.calculate_physical_coordinates(R1, T1, Q1, scale_factor_A)
        P2 = self.calculate_physical_coordinates(R2, T2, Q2, scale_factor_B)

        # Fusion: X from B, Y and Z from A
        fused_coordinates = np.column_stack((P2[:, 0], P1[:, 1], P1[:, 2]))
        fused_coordinates_mean = np.column_stack((P2[:, 0], (P1[:, 1] + P2[:, 1]) / 2, P1[:, 2]))

        return fused_coordinates, fused_coordinates_mean




In [3]:
# Usage Example
image_A = cv2.imread('/content/T2PA1.jpg')
image_B = cv2.imread('/content/T2PB1.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[140.04030528 121.01805715 226.95702158]
 [129.52398679 105.42029189 201.10256918]
 [110.49139709 103.39390424 245.28229098]
 [143.7141595   40.86953158 198.58426171]
 [111.44949213  37.89755448 254.77525645]
 [123.29759421 -53.25220354 233.77406425]
 [ 60.17585923 -30.65814277 241.37097981]]
Fused 3D Coordinates (Mean):
 [[140.04030528 128.05394085 226.95702158]
 [129.52398679 109.8454971  201.10256918]
 [110.49139709 108.77976112 245.28229098]
 [143.7141595   44.96679751 198.58426171]
 [111.44949213  36.59279459 254.77525645]
 [123.29759421 -59.42325929 233.77406425]
 [ 60.17585923 -46.44150933 241.37097981]]


# Calculating physical coordinates for 7 joints using 20 static images

In [4]:
# Usage Example
image_A = cv2.imread('/content/T2PA1.jpg')
image_B = cv2.imread('/content/T2PB1.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[140.04030528 121.01805715 226.95702158]
 [129.52398679 105.42029189 201.10256918]
 [110.49139709 103.39390424 245.28229098]
 [143.7141595   40.86953158 198.58426171]
 [111.44949213  37.89755448 254.77525645]
 [123.29759421 -53.25220354 233.77406425]
 [ 60.17585923 -30.65814277 241.37097981]]
Fused 3D Coordinates (Mean):
 [[140.04030528 128.05394085 226.95702158]
 [129.52398679 109.8454971  201.10256918]
 [110.49139709 108.77976112 245.28229098]
 [143.7141595   44.96679751 198.58426171]
 [111.44949213  36.59279459 254.77525645]
 [123.29759421 -59.42325929 233.77406425]
 [ 60.17585923 -46.44150933 241.37097981]]


In [5]:
image_A = cv2.imread('/content/T2PA2.jpg')
image_B = cv2.imread('/content/T2PB2.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[186.8939061  120.53434113 223.07662844]
 [168.13366652 103.15302043 198.29148892]
 [161.73845946 101.96705989 246.13774128]
 [176.28264232  32.14049287 194.21605065]
 [169.63925861  31.66303224 256.18543685]
 [124.22457333 -52.72423831 233.36054191]
 [195.00152775 -61.61443024 241.50316339]]
Fused 3D Coordinates (Mean):
 [[186.8939061  128.44818236 223.07662844]
 [168.13366652 108.26682652 198.29148892]
 [161.73845946 108.87644684 246.13774128]
 [176.28264232  39.89510699 194.21605065]
 [169.63925861  32.89891855 256.18543685]
 [124.22457333 -57.69716188 233.36054191]
 [195.00152775 -67.59417865 241.50316339]]


In [6]:
image_A = cv2.imread('/content/T2PA3.jpg')
image_B = cv2.imread('/content/T2PB3.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[252.3961195  123.70977225 240.18209667]
 [226.32493135 106.62999214 209.59098044]
 [229.79216295 102.44667402 262.14385024]
 [228.40648912  28.75555744 206.04104626]
 [235.68147731  28.36331368 269.12317416]
 [253.77175496 -79.14270134 229.41431335]
 [192.45303436 -67.72030678 242.15731317]]
Fused 3D Coordinates (Mean):
 [[252.3961195  128.52930717 240.18209667]
 [226.32493135 109.61169592 209.59098044]
 [229.79216295 106.32342967 262.14385024]
 [228.40648912  34.34441333 206.04104626]
 [235.68147731  28.64335397 269.12317416]
 [253.77175496 -73.09308509 229.41431335]
 [192.45303436 -68.9948865  242.15731317]]


In [7]:
image_A = cv2.imread('/content/T2PA4.jpg')
image_B = cv2.imread('/content/T2PB4.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[308.02016157 126.6134528  217.20127843]
 [285.61718368 103.07108506 187.47817456]
 [287.62941764 104.53587195 243.14284484]
 [286.05400277  17.87779205 185.79455156]
 [293.70997138  22.34675592 258.76434982]
 [254.51873232 -92.29676237 233.0333132 ]
 [316.7067456  -92.64419291 247.98349171]]
Fused 3D Coordinates (Mean):
 [[308.02016157 128.95228058 217.20127843]
 [285.61718368 107.80237534 187.47817456]
 [287.62941764 108.43914909 243.14284484]
 [286.05400277  29.15353258 185.79455156]
 [293.70997138  27.71231252 258.76434982]
 [254.51873232 -77.84162928 233.0333132 ]
 [316.7067456  -81.98567203 247.98349171]]


In [8]:
image_A = cv2.imread('/content/T2PA5.jpg')
image_B = cv2.imread('/content/T2PB5.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[ 370.99410929  135.70848313  236.09194285]
 [ 342.18843259  109.9848812   202.24535726]
 [ 350.96187054  106.02634695  265.04096776]
 [ 346.99773634   16.29143524  194.31054807]
 [ 359.95391791   13.95284207  275.82310771]
 [ 371.45100655 -122.89387761  225.67620472]
 [ 318.9027272  -108.07808745  243.75566803]]
Fused 3D Coordinates (Mean):
 [[370.99410929 136.11591473 236.09194285]
 [342.18843259 114.16300063 202.24535726]
 [350.96187054 110.88476406 265.04096776]
 [346.99773634  28.87611124 194.31054807]
 [359.95391791  23.19675653 275.82310771]
 [371.45100655 -93.83704434 225.67620472]
 [318.9027272  -89.70711434 243.75566803]]


In [9]:
image_A = cv2.imread('/content/T2PA6.jpg')
image_B = cv2.imread('/content/T2PB6.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[ 420.78467568  151.55154175  211.56055684]
 [ 389.77028141  110.59370503  173.6574017 ]
 [ 404.02842913  110.91160194  246.0123771 ]
 [ 398.05355933   -2.24687636  173.90454091]
 [ 412.47447781    2.50204188  262.70919399]
 [ 374.56647553 -142.19642361  229.3929835 ]
 [ 431.25854142 -147.39496087  246.76666442]]
Fused 3D Coordinates (Mean):
 [[ 420.78467568  145.76554096  211.56055684]
 [ 389.77028141  114.64451756  173.6574017 ]
 [ 404.02842913  112.85276431  246.0123771 ]
 [ 398.05355933   19.41094636  173.90454091]
 [ 412.47447781   17.36384989  262.70919399]
 [ 374.56647553 -102.94568993  229.3929835 ]
 [ 431.25854142 -110.23604227  246.76666442]]


In [10]:
image_A = cv2.imread('/content/T2PA7.jpg')
image_B = cv2.imread('/content/T2PB7.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[ 481.71042906  153.34355734  227.65270305]
 [ 445.94556641  113.48590027  184.62858776]
 [ 463.51192085  113.11549911  267.48277487]
 [ 453.11961591  -17.32734133  179.1634273 ]
 [ 472.68247164  -13.51063011  282.58995344]
 [ 460.57425645 -205.10401586  219.10050732]
 [ 432.91077184 -157.20833565  242.85458414]]
Fused 3D Coordinates (Mean):
 [[ 481.71042906  144.98132423  227.65270305]
 [ 445.94556641  115.33512226  184.62858776]
 [ 463.51192085  114.93096283  267.48277487]
 [ 453.11961591    9.74497998  179.1634273 ]
 [ 472.68247164    9.41727315  282.58995344]
 [ 460.57425645 -134.75854053  219.10050732]
 [ 432.91077184 -114.50560632  242.85458414]]


In [11]:
image_A = cv2.imread('/content/T2PA8.jpg')
image_B = cv2.imread('/content/T2PB8.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Fused 3D Coordinates (Normal):
 [[ 527.46304239  166.50500177  202.20741473]
 [ 491.69668139  114.51070759  149.29316793]
 [ 511.27452061  113.7628295   251.57690598]
 [ 501.45068961  -41.01173719  147.58575394]
 [ 522.65079227  -38.48640523  273.12344519]
 [ 476.48763129 -211.55158029  217.72883347]
 [ 531.36919105 -228.2801356   245.31274756]]
Fused 3D Coordinates (Mean):
 [[ 5.27463042e+02  1.51834397e+02  2.02207415e+02]
 [ 4.91696681e+02  1.15936878e+02  1.49293168e+02]
 [ 5.11274521e+02  1.15156861e+02  2.51576906e+02]
 [ 5.01450690e+02  1.82871069e-01  1.47585754e+02]
 [ 5.22650792e+02 -1.98338670e+00  2.73123445e+02]
 [ 4.76487631e+02 -1.38929904e+02  2.17728833e+02]
 [ 5.31369191e+02 -1.49602540e+02  2.45312748e+02]]


In [12]:
image_A = cv2.imread('/content/T2PA9.jpg')
image_B = cv2.imread('/content/T2PB9.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Landmarks not found.


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 0 and the array at index 1 has size 7

In [ ]:
### for static image 9 and 10 we getting error, because in the image whole body is not visible so mediapipe not able to detect points.
### we need good cameras to capture images in better way.



In [14]:
image_A = cv2.imread('/content/T2PA10.jpg')
image_B = cv2.imread('/content/T2PB10.jpg')

pose_fusion = PoseFusion(image_A, image_B)
fused_coords, fused_coords_mean = pose_fusion.fuse_coordinates()

if fused_coords is not None:
    print("Fused 3D Coordinates (Normal):\n", fused_coords)
    print("===============")
    print("Fused 3D Coordinates (Mean):\n", fused_coords_mean)

Landmarks not found.


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 0 and the array at index 1 has size 7